# Arbeiten mit JSON-Daten und einer SQLite-Datenbank

In diesem Notebook werden wir semi-strukturierte JSON-Daten in Python laden und in eine relationale SQLite-Datenbank konvertieren. Anschließend führen wir verschiedene SQL-Abfragen durch, um die Daten zu analysieren und Erkenntnisse zu gewinnen.
Schritte:

    Laden der JSON-Daten und Konvertierung in ein Pandas DataFrame.
    Erstellen einer In-Memory-SQLite-Datenbank und Import der Daten.
    Ausführen von SQL-Abfragen, um Daten abzurufen und zu analysieren.

***Schritt 1:*** JSON-Daten laden und in ein Pandas DataFrame konvertieren

Zunächst erstellen wir ein Beispiel für JSON-Daten, die Benutzerdaten und deren Aktivitäten enthalten. Wir werden diese JSON-Daten in Python laden und in ein Pandas DataFrame konvertieren, um die Daten besser zu visualisieren und zu bearbeiten.

In [1]:
import json
import pandas as pd

# Example of JSON data (semi-structured)
json_data = '''
[
    {
        "user_id": 1,
        "username": "PlayerOne",
        "email": "playerone@example.com",
        "activities": [
            {"game": "GameA", "score": 1500, "date": "2024-08-01"},
            {"game": "GameB", "score": 3000, "date": "2024-08-05"}
        ]
    },
    {
        "user_id": 2,
        "username": "GamerGirl",
        "email": "gamergirl@example.com",
        "activities": [
            {"game": "GameA", "score": 2000, "date": "2024-08-03"}
        ]
    }
]
'''

# Load the JSON data
data = json.loads(json_data)

# Convert the list of dictionaries into a Pandas DataFrame
df = pd.DataFrame(data)


#print("Tabular Data:")
print(df)


   user_id   username                  email  \
0        1  PlayerOne  playerone@example.com   
1        2  GamerGirl  gamergirl@example.com   

                                          activities  
0  [{'game': 'GameA', 'score': 1500, 'date': '202...  
1  [{'game': 'GameA', 'score': 2000, 'date': '202...  


***Schritt 2:*** Erstellen einer In-Memory-SQLite-Datenbank

Jetzt erstellen wir eine SQLite-Datenbank im Arbeitsspeicher (In-Memory) und importieren die JSON-Daten in zwei Tabellen: Users und Activities.

In [2]:
import sqlite3

# Connect to an in-memory SQLite database
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

# Create the Users table
cursor.execute('''
CREATE TABLE Users (
    user_id INTEGER PRIMARY KEY,
    username TEXT,
    email TEXT
)
''')

# Create the Activities table
cursor.execute('''
CREATE TABLE Activities (
    activity_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER,
    game TEXT,
    score INTEGER,
    date DATE,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
)
''')

# Insert  data into the relational database
cursor.execute('''
INSERT INTO Users (user_id, username, email) VALUES (1, 'PlayerOne', 'playerone@example.com')
''')

cursor.execute('''
INSERT INTO Users (user_id, username, email) VALUES (2, 'GamerGirl', 'gamergirl@example.com')
''')

# Direkteingabe von Daten in die Activities-Tabelle
cursor.execute('''
INSERT INTO Activities (user_id, game, score, date) VALUES (1, 'GameA', 1500, '2024-08-01')
''')

cursor.execute('''
INSERT INTO Activities (user_id, game, score, date) VALUES (1, 'GameB', 3000, '2024-08-05')
''')

cursor.execute('''
INSERT INTO Activities (user_id, game, score, date) VALUES (2, 'GameA', 2000, '2024-08-03')
''')# Commit the changes
connection.commit()


***Schritt 3:*** Abfragen der Daten in der SQLite-Datenbank

Nun führen wir verschiedene SQL-Abfragen durch, um die in der Datenbank gespeicherten Informationen abzurufen und zu analysieren.

Abfrage 1: Alle Daten aus der Users-Tabelle anzeigen

In [3]:
# Query to check all data in the Users table
cursor.execute('SELECT * FROM Users')
rows = cursor.fetchall()

print("Users Table Output:")
for row in rows:
    print(row)

Users Table Output:
(1, 'PlayerOne', 'playerone@example.com')
(2, 'GamerGirl', 'gamergirl@example.com')


Abfrage 2: Daten für einen bestimmten User anzeigen

In [4]:
# Query to check data for a specific user
cursor.execute('''
SELECT * FROM Users WHERE username = "PlayerOne"
''')
rows = cursor.fetchall()

print("\nData for PlayerOne:")
for row in rows:
    print(row)


Data for PlayerOne:
(1, 'PlayerOne', 'playerone@example.com')


Abfrage 3: Join der Tabellen Users und Activities für einen bestimmten User

In [5]:
# This query joins the Users and Activities tables and filters results where the username is 'PlayerOne'
cursor.execute('''
SELECT Users.user_id, Users.username, Users.email, Activities.game, 
Activities.score, Activities.date
FROM Activities
JOIN Users ON Activities.user_id = Users.user_id
WHERE Users.username = 'PlayerOne';
''')

# The pure SQL code would be:
# SELECT Users.user_id, Users.username, Users.email, Activities.game, 
# Activities.score, Activities.date
# FROM Activities
# JOIN Users ON Activities.user_id = Users.user_id
# WHERE Users.username = 'PlayerOne';

# Fetch and print the results
rows = cursor.fetchall()

print("Joined Data for PlayerOne:")
for row in rows:
    print(row)

Joined Data for PlayerOne:
(1, 'PlayerOne', 'playerone@example.com', 'GameA', 1500, '2024-08-01')
(1, 'PlayerOne', 'playerone@example.com', 'GameB', 3000, '2024-08-05')


Abfrage 4: Anzahl der einzigartigen User in der Datenbank zählen

In [6]:
# Query to count the number of unique players (distinct users) in the database
cursor.execute('''
SELECT COUNT(DISTINCT user_id) 
FROM Users;
''')

# Fetch and print the number of unique players
num_unique_players = cursor.fetchone()[0]
print(f"Number of unique players in the database: {num_unique_players}")

Number of unique players in the database: 2


Abfrage 5: Durchschnittliche Punktzahl für PlayerOne berechnen

In [7]:
# Calculate the average score for PlayerOne
cursor.execute('''
SELECT AVG(score) 
FROM Activities
JOIN Users ON Activities.user_id = Users.user_id
WHERE Users.username = 'PlayerOne';
''')


Abfrage 6: Durchschnittliche Punktzahl für alle Benutzer berechnen

In [8]:
# Fetch and print the average score for PlayerOne
avg_score_playerone = cursor.fetchone()[0]
print(f"Average Score for PlayerOne: {avg_score_playerone}")

# Calculate the average score across all users
cursor.execute('''
SELECT AVG(score) 
FROM Activities;
''')

# Fetch and print the average score across all users
avg_score_all_users = cursor.fetchone()[0]
print(f"Average Score across all users: {avg_score_all_users}")

Average Score for PlayerOne: 2250.0
Average Score across all users: 2166.6666666666665


Daten in der relationalen Datenbank anzeigen

In [9]:
# Query to check the data in the relational database
cursor.execute('''
SELECT Users.user_id, Users.username, Users.email, Activities.game, 
Activities.score, Activities.date
FROM Activities
JOIN Users ON Activities.user_id = Users.user_id;
''')

rows = cursor.fetchall()

print("\nRelational Database Output:")
for row in rows:
    print(row)



Relational Database Output:
(1, 'PlayerOne', 'playerone@example.com', 'GameA', 1500, '2024-08-01')
(1, 'PlayerOne', 'playerone@example.com', 'GameB', 3000, '2024-08-05')
(2, 'GamerGirl', 'gamergirl@example.com', 'GameA', 2000, '2024-08-03')


Verbindung schließen

In [10]:

# Close the connection
connection.close()